# Cluster the Olivetti Faces Dataset with K-Means

(Notebook adapted from: A.Geron, Hands On ML with Scikit-Learn, Keras und Tensorflow, O'Reilly)

In this exercise we will try to cluster human pictures, in order to identify which picture belong to the same subject/person. 
We will use an unsupervised learning approach, which means we will train without knowing the correct assignment of the pictures (in other words the person have no labels).
As usual, answer the questions/comments and add your code where indicated. Otherwise just try to understand what's happening :-).

About the database: The classic Olivetti faces dataset contains 400 grayscale 64 × 64–pixel images of faces. Each image is flattened to a 1D vector of size 4,096. 40 different people were photographed (10 times each). Load the dataset using the `sklearn.datasets.fetch_olivetti_faces()` function.
For some subjects, the images were taken at different times, varying the lighting, facial expressions (open / closed eyes, smiling / not smiling) and facial details (glasses / no glasses). All the images were taken against a dark
homogeneous background with the subjects in an upright, frontal position (with tolerance for some side movement).

## Libraries and settings

In [ ]:
# Libraries
import os
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.datasets import fetch_olivetti_faces

# Settings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

## Import data

In [ ]:
# Import data
olivetti = fetch_olivetti_faces()

# Show data description
# print(olivetti.DESCR) 

# Data matrix
X = olivetti.data

# Data shape (nrows, ncols)
X.shape

## Reduce dimensionality

Since the images are 64x64pixels, the number of features (=number of total pixels) is 4096. This a huge number of variables to check out for clustering. So, for convenience we should first try to see if we can reduce the number of features we feed to k-means. This operation is called "Dimensionality Reduction".

In [ ]:
# Use PCA to reduce the dimensionality of the data
pca = PCA(0.99)

# Fit the PCA model to the data
X_pca = pca.fit_transform(olivetti.data)

# Check the number of components
pca.n_components_

## Perform k-means clustering

Well done :) we reduced dimensionality quite a lot!

Let's now perform k-means on this subset of features: X_pca  

Use the function https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html and the advices given in the class.  

Remember that you have to define the number of clusters k.

How to set k? Start with a "reasonable" guess and qualitatively look how good the clustering is. 

Try to change k manually and see if it improves or get worse.

In [ ]:
# Perform k-means clustering
kmeans = KMeans(n_clusters=5, random_state=42) ### ADD YOUR CODE HERE ###
kmeans.fit(X_pca)

def plot_faces(faces, n_cols=5):
    n_rows = (len(faces) - 1) // n_cols + 1
    plt.figure(figsize=(n_cols, n_rows * 1.1))
    for index, face in enumerate(faces):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(face.reshape(64, 64), cmap="gray")
        plt.axis("off")
    plt.show()

for cluster_id in np.unique(kmeans.labels_):
    print("Cluster", cluster_id)
    in_cluster = kmeans.labels_==cluster_id
    faces = X[in_cluster].reshape(-1, 64, 64)
    plot_faces(faces)

What was your best k?

## Use a loop to identify the best k

For a serious approach we should try to identify K through a more robust approach. 
Let's run the k-means algorithm in a loop, every time with a different number of clusters, K. 

In [ ]:
 # Feel free to change the range limits or step
k_range = range(5, 150, 5)
kmeans_per_k = []
for k in k_range:
    print("k={}".format(k))
    kmeans = KMeans(n_clusters=k)  ### ADD YOUR CODE HERE ###
    kmeans.fit(X_pca)
    kmeans_per_k.append(kmeans)

## Calculate Intertia (Within-Cluster Sum of Squares, WCSS)  for each run

Let's now evaluate for each of this run the inertia of the model. Luckily, this is part of the Sklearn KMeans function's output (Z.B. kmeans.inertia_) 

In [ ]:
# Calculate inertia for each k
inertias = [model.inertia_ for model in kmeans_per_k]

plt.figure(figsize=(7, 4))
plt.plot(k_range, inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.grid()
plt.show()

Can you get from this plot a clear indication on what could be a good value for k?  

Do you see an "elbow"?

## Calculate silhouette score

Let's try with the more sophisticated method called silhouette analysis.

In [ ]:
# Calculate silhouette score for each k
silhouette_scores = [silhouette_score(X_pca, model.labels_) for model in kmeans_per_k]
best_index = np.argmax(silhouette_scores)
best_k = k_range[best_index]
best_score = silhouette_scores[best_index]

plt.figure(figsize=(7, 4))
plt.plot(k_range, silhouette_scores, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Silhouette score", fontsize=14)
plt.plot(best_k, best_score, "rs")
plt.grid()
plt.show()

What is the best value for k according to this method?

In [ ]:
print(f"The best k is: {best_k}")

It looks like the best number of clusters is quite high. 
How does it compare with your first reasonable guess? 

## Final result

Let's have a direct look of how pictures are grouped according to the best k

In [ ]:
# Fit the best k-means model
kmeans = KMeans(n_clusters = 140) ### ADD VALUE OF BEST K HERE ###
kmeans.fit(X_pca)

In [ ]:
# Plot the faces in each cluster
def plot_faces(faces, n_cols=5):
    n_rows = (len(faces) - 1) // n_cols + 1
    plt.figure(figsize=(n_cols, n_rows * 1.1))
    for index, face in enumerate(faces):
        plt.subplot(n_rows, n_cols, index + 1)
        plt.imshow(face.reshape(64, 64), cmap="gray")
        plt.axis("off")
    plt.show()

for cluster_id in np.unique(kmeans.labels_):
    print("Cluster", cluster_id)
    in_cluster = kmeans.labels_==cluster_id
    faces = X[in_cluster].reshape(-1, 64, 64)
    plot_faces(faces)

Well done :-)

### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [ ]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')